In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
from math import log
from PIL import ImageEnhance
import cv2
import matplotlib.cm
from PIL.ImageColor import getrgb
from scipy.interpolate import interp1d
np.warnings.filterwarnings("ignore")

In [2]:
class MandelbrotSet:
    max_iterations: int
    escape_radius: float = 2.0

    def __init__(self, max_iterations, escape_radius):
        self.max_iterations = max_iterations 
        self.escape_radius = escape_radius

    def __contains__(self, c: complex) -> bool:
        return self.stability(c) == 1
    
    def stability(self, c: complex, smooth=False, clamp=True) -> float:
        value = self.escape_count(c, smooth) / self.max_iterations
        return max(0.0, min(value, 1.0)) if clamp else value
    
    def escape_count(self, c: complex, smooth=False) -> float:
        z = 0
        for iteration in range(self.max_iterations):
            z = z ** 2 + c
            if abs(z) > self.escape_radius:
                if smooth:
                    return iteration + 1 - log(log(abs(z))) / log(2)
                return iteration
        return self.max_iterations


In [3]:
from viewport import Viewport

In [4]:
def hsb(hue_degrees: int, saturation: float, brightness: float):
    return getrgb(
        f"hsv({hue_degrees % 360},"
        f"{saturation * 100}%,"
        f"{brightness * 100}%)"
    )

In [5]:
def paint(mandelbrot_set, viewport, palette, smooth):
    for pixel in viewport:
        stability = mandelbrot_set.stability(complex(pixel), smooth)
        pixel.color = (0, 0, 0) if stability == 1 else hsb(
            hue_degrees=int(stability * 360*1),
            saturation=stability,
            brightness=1,
        )

In [25]:
mapping = [(66, 30, 15),
 (25, 7, 26),
 (9, 1, 47),
 (4, 4, 73),
 (0, 7, 100),
 (12, 44, 138),
 (24, 82, 177),
 (57, 125, 209),
 (134, 181, 229),
 (211, 236, 248),
 (241, 233, 191),
 (248, 201, 95),
 (255, 170, 0),
 (204, 128, 0),
 (153, 87, 0),
 (106, 52, 3)]

In [8]:
def newPaint(mandelbrot_set, viewport, mapping, smooth, iterations):
    for pixel in viewport:
        stability = mandelbrot_set.stability(complex(pixel), smooth)
        layer = stability*iterations
        pixel.color = (0, 0, 0) if stability == 1 else mapping[int(layer % 16)]
        

In [9]:
def denormalize(palette):
    return [
        tuple(int(channel * 255) for channel in color)
        for color in palette
    ]

In [10]:
def make_gradient(colors, interpolation="linear"):
    X = [i / (len(colors) - 1) for i in range(len(colors))]
    Y = [[color[i] for color in colors] for i in range(3)]
    channels = [interp1d(X, y, kind=interpolation) for y in Y]
    return lambda x: [np.clip(channel(x), 0, 1) for channel in channels]

In [11]:
black = (0, 0, 0)
blue = (0, 0, 1)
maroon = (0.5, 0, 0)
navy = (0, 0, 0.5)
red = (1, 0, 0)

colors = [black, navy, blue, maroon, red, black]
gradient = make_gradient(colors, interpolation="cubic")

In [12]:
max_iterations = 1000
mandelbrot_set = MandelbrotSet(max_iterations=int(max_iterations), escape_radius=1000)
unitRange = [3 * (1 - 1/(2**3))**x for x in range(250)]
imageList = []
width, height = 512, 512
center = -0.77679088889099 + 0.135781j
# center = -0.7756837680090538 + 0.13646736829469008j
# center=-0.7435 + 0.1314j



num_colors = 256
palette = denormalize([
    gradient(i / num_colors) for i in range(num_colors)
])

for unit in unitRange:
    image = Image.new(mode="RGB", size=(width, height))
    viewport = Viewport(image, center=center, width=unit)
    # paint(mandelbrot_set, viewport, palette, smooth=True)
    newPaint(mandelbrot_set, viewport, mapping, True, max_iterations)
    imageList.append(image)

# imageList[-1].show()

In [13]:
unitRange[-1:]

[1.0892472126816586e-14]

In [14]:
imageList[0].save("images/Mandelbrot.png")

In [15]:
video = cv2.VideoWriter("images/Mandelbrot.mp4",-1,10,(width,height))

try:
    for i in range(len(imageList)):
        video.write(cv2.cvtColor(np.array(imageList[i]), cv2.COLOR_RGB2BGR))
finally: 
    video.release()

In [16]:
#Save as gif
frame_one = imageList[0]
frame_one.save("images/example.gif", format="GIF", append_images=imageList[1:],
               save_all=True, duration=1, loop=0)

In [17]:
[2 * (1 - 1/(2**3))**x for x in range(100)]

[2.0,
 1.75,
 1.53125,
 1.33984375,
 1.17236328125,
 1.02581787109375,
 0.8975906372070312,
 0.7853918075561523,
 0.6872178316116333,
 0.6013156026601791,
 0.5261511523276567,
 0.46038225828669965,
 0.4028344760008622,
 0.3524801665007544,
 0.3084201456881601,
 0.2698676274771401,
 0.2361341740424976,
 0.2066174022871854,
 0.18079022700128722,
 0.15819144862612633,
 0.13841751754786052,
 0.12111532785437797,
 0.10597591187258072,
 0.09272892288850812,
 0.08113780752744461,
 0.07099558158651403,
 0.06212113388819978,
 0.054355992152174806,
 0.04756149313315296,
 0.04161630649150884,
 0.03641426818007023,
 0.03186248465756145,
 0.027879674075366272,
 0.024394714815945486,
 0.0213453754639523,
 0.018677203530958263,
 0.01634255308958848,
 0.014299733953389921,
 0.012512267209216181,
 0.010948233808064159,
 0.009579704582056139,
 0.00838224150929912,
 0.007334461320636731,
 0.0064176536555571395,
 0.005615446948612497,
 0.004913516080035935,
 0.004299326570031443,
 0.003761910748777513,
 0